In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from keras import Input

In [2]:
with open('./data/img193.pkl', 'rb') as f:
    img193 = pickle.load(f)

with open('./data/img304.pkl', 'rb') as f:
    img304 = pickle.load(f)

with open('./data/value.pkl', 'rb') as f:
    value = pickle.load(f)

In [8]:
img193_train, img193_test, img304_train, img304_test, value_train, value_test = train_test_split(img193, img304, value, test_size=0.2, random_state=42)

MemoryError: Unable to allocate 4.37 GiB for an array with shape (2239, 512, 512, 1) and data type float64

In [5]:
input_shape = (512, 512, 1)

In [7]:
model = Sequential()
model.add(Conv2D(256, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='linear'))

model.compile(loss='mse', optimizer='adam', metrics=['mae'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 510, 510, 256)     2560      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 255, 255, 256)    0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 253, 253, 128)     295040    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 126, 126, 128)    0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 124, 124, 64)      73792     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 62, 62, 64)      

In [ ]:
from tensorflow.keras.layers import Input, concatenate
from tensorflow.keras.models import Model

def create_cnn_branch(input_shape):
    input_layer = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), activation='relu')(input_layer)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    return Model(inputs=input_layer, outputs=x)


In [ ]:
input_shape = (512, 512, 1)  # Shape of each input image

# Create two CNN branches
cnn_branch1 = create_cnn_branch(input_shape)
cnn_branch2 = create_cnn_branch(input_shape)

# Image inputs
input_img193 = Input(shape=input_shape)
input_img304 = Input(shape=input_shape)

# Process each image
processed_img193 = cnn_branch1(input_img193)
processed_img304 = cnn_branch2(input_img304)

# Concatenate the outputs
concatenated = concatenate([processed_img193, processed_img304])

# Fully connected layers and regression output
x = Dense(512, activation='relu')(concatenated)
x = Dense(256, activation='relu')(x)
output = Dense(1, activation='linear')(x)  # Single float output

# Complete model
model = Model(inputs=[input_img193, input_img304], outputs=output)

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit([img193_, img304], y_train, epochs=num_epochs, batch_size=batch_size)